In [1]:
import pandas as pd
import numpy as np

np.random.seed(42)

data = pd.DataFrame({
    "transaction_id": range(1, 1001),
    "age": np.random.randint(18, 28, 1000),
    "transaction_amount": np.random.randint(100, 20000, 1000),
    "transaction_time": np.random.randint(0, 24, 1000),
    "location_change": np.random.choice([0,1], 1000, p=[0.85,0.15]),
    "is_fraud": np.random.choice([0,1], 1000, p=[0.9,0.1])
})

data.head()

,transaction_id,age,transaction_amount,transaction_time,location_change,is_fraud
0,1,24,15428,15,0,1
1,2,21,927,12,0,0
2,3,25,7247,22,0,0
3,4,22,4551,18,1,0
4,5,24,15443,7,0,0


In [2]:
import sqlite3

conn = sqlite3.connect("genz_fraud.db")
data.to_sql("transactions", conn, if_exists="replace", index=False)

1000

In [4]:
query = """
SELECT AVG(transaction_amount) as avg_amount
FROM transactions
WHERE is_fraud = 1
"""
pd.read_sql(query, conn)

,avg_amount
0,9142.181818


In [5]:
query2 = """
SELECT COUNT(*) as night_frauds
FROM transactions
WHERE is_fraud = 1 AND transaction_time > 22
"""
pd.read_sql(query2, conn)

,night_frauds
0,3


In [6]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

X = data.drop(["transaction_id","is_fraud"], axis=1)
y = data["is_fraud"]

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2)

model = LogisticRegression()
model.fit(X_train, y_train)

predictions = model.predict(X_test)

print("Accuracy:", accuracy_score(y_test, predictions))

Accuracy: 0.885


/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
